In [1]:
import csv
import numpy as np
import ast
import json
import os
import sys
import re
from textblob import TextBlob
import collections
import pandas as pd
import tldextract
import requests
from urllib.parse import urlparse, urlsplit, urlunsplit
from time import sleep

In [2]:
def featuresExtraction(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    features = []
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    
    return np.array(features)

In [3]:
def labelExtraction(label_file):
#     csv_file = open(file)
#     csv_reader = csv.reader(csv_file, delimiter = ' ')
#     labels = []
#     for row in csv_reader:
#         labels.append(row[0])
    labels = np.genfromtxt(label_file,dtype=None)
    return labels

In [4]:
def reduction(array, key):
    reducedArray = []
    for i, j in enumerate(array):
        if len(reducedArray) == 0:
            reducedArray.append(j)
        else:
            count = 0
            for k in range(len(reducedArray)):
                if a[k] != b[i]:
                    count = count + 1
            
                if count == len(a):
                    a.append(b[i])
    
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    return reducedArray

In [5]:
def getCredentials():
    app_key = "jGVy1ilV55A7P24eJHuousM1b"
    app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
    oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
    oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
    
    return app_key, app_secret, oauth_token, oauth_token_secret

In [6]:
def featuresExtraction2(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    final = []
    textFeatures = []
    
    for i, j in enumerate(convertedArray):
        tempDict = {}
        tempDict['id'] = j['id']
        tempDict['date'] = j['created_at']
        tempDict['username'] = j['user']['screen_name']
        tempDict['hashtags'] = j['entities']['hashtags']
        tempDict['user_mentions'] = j['entities']['user_mentions']
        tempDict['favorites'] = j['favorite_count']
        
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            textFeatures.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            textFeatures.append(j['extended_tweet']['full_text'])
        else:
            textFeatures.append(j['text'])
        
        final.append(tempDict)
    
    return final, textFeatures

In [7]:
def authorization(key):
    if key == 'e' or 'E':
        app_key = "jGVy1ilV55A7P24eJHuousM1b"
        app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
        oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
        oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
        return app_key, app_secret, oauth_token, oauth_token_secret
    elif key == 'v' or 'V':
        app_key = "d"
    else:
        print("Wrong identification key")

In [8]:
def cred_Processing(tweetsFile):
    idRegEx = re.compile(r".*ID=")
    endElRegEX = re.compile(r"'.*")
    output = open("outputTweets.json", "w")
    
    with open(tweetsFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            
            topicKey = topicData[0]
            topicTerms = topicData[1]
            topicTweetCount = topicData[2]
            tweetIdList = topicData[3]
            
            print(topicKey)
            
            realTweetsIds = []
            
            idElements = tweetIdList.split("),")
            for element in idElements:
                elArr = element.split(",")
                idEl = list(filter(lambda x: "ID" in x, elArr))[0]
                idEl = idRegEx.sub("", idEl)
                idEl = endElRegEX.sub("", idEl)
                
                realTweetsIds.append(int(idEl))
                
            realTweetsIds = list(set(realTweetsIds))
            
            topicMap = {
                "key" : topicKey,
                "terms" : topicTerms.split(","),
                "count" : topicTweetCount, 
                "tweets" : realTweetsIds
            }
            
            json.dump(topicMap, output, sort_keys=True)
            output.write("\n")
    output.close()

In [18]:
def rating_processing(ratingFile, tweetsFile):
    tweetsMap = {}
    
    with open(tweetsFile, "r") as f:
        for line in f:
            try:
                tweetData = json.loads(line)
                tweetsMap[tweetData["key"]] = tweetData
            except:
                print("Error")
            
    output = open(r"/media/eric/TOSHIBA EXT/University/Research/Data/CREDBANK/finalOutput.json", "w")
    
    with open(ratingFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            topicKey = topicData[0]
            topicTerms = topicData[1]
            ratings = topicData[2]
            reasons = topicData[3]
            
            try:
                ratings = list(map(lambda x: int(x.strip().replace("'", "")), ratings.replace("[", "").replace("]", "").split(",")))
                ratings = np.array(ratings)
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Inserting ratings for {}".format(topicKey))
                tweetsMap[topicKey]["ratings"] = ratings.tolist()
                print("Success")
                print("Inserting mean for {}".format(topicKey))
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Success")
                topicMap = tweetsMap[topicKey]
                print(topicMap["key"], topicMap["mean"])
                json.dump(topicMap, output, sort_keys=True)
                output.write("\n")
            except:
                print("{} is missing.".format(topicKey))
            
            
            
    output.close()

In [10]:
def binaryConversion(outputFile):
    labels = {}
    with open(outputFile, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            if(tweet['mean'] >= 1.9):
                labels[tweet['key']] = 1
            elif(tweet['mean'] <= 1.467):
                labels[tweet['key']] = 0
    return labels

In [11]:
#TODO: Retweet and date
def singleMessageFeatureExtraction(tweet_json):
    #print(tweet_json)
    retweets = 0
    try:
        retweets = tweet_json["tweet"]["retweet_status"]
        retweets = 1
    except:
        retweets = 0
    date = tweet_json["tweet"]["created_at"].split(" ")[0]
    dates = []
    if date == "Mon":
        dates = [1,0,0,0,0,0,0]
    elif date == "Tue":
        dates = [0,1,0,0,0,0,0]
    elif date == "Wed":
        dates = [0,0,1,0,0,0,0]
    elif date == "Thu":
        dates = [0,0,0,1,0,0,0]
    elif date == "Fri":
        dates = [0,0,0,0,1,0,0]
    elif date == "Sat":
        dates = [0,0,0,0,0,1,0]
    elif date == "Sun":
        dates = [0,0,0,0,0,0,1]
    return retweets, dates

In [12]:
def unshorten_url(url):
    return requests.head(url, allow_redirects=True).url

In [3]:
def messageFeatureExtraction(tweets, tweet_json, threshold):
    tweetsFeatures = None
    np.array(tweetsFeatures)
    count = 0
    for topic in topics:
        print("Starting feature extraction for topic {} of {}".format(count, len(topics)))
        indiTweetFeature = []
        
        #Number of tweets
        indiTweetFeature.append(len(topic))
        

        for tweet in topic:

            #Length of characters 1
            print("Adding Length of charactrs")
            indiTweetFeature.append(len(tweet))
        
            print("Adding words")
            #Length of words 2
            indiTweetFeature.append(len(tweet.split(" ")))
        
            print("Adding question mark")
            #Contains question mark 3
            results = collections.Counter(tweet)
            if(results['?'] > 1):
                indiTweetFeature.append(1)
            else:
                indiTweetFeature.append(0)

            print('Adding exclamation mark')
            #Contains exclamation 4
            if(results['!'] > 1):
                indiTweetFeature.append(1)
            else:
                indiTweetFeature.append(0)

            print("Adding smiles")
            #Contains emoticon smile 5
            emojis = "😊"
            indiTweetFeature.append(tweet.count(emojis))

            print("!?")
            #Contains multiple question or exclamation marks 6
            if results['?'] and results['!'] > 1:
                indiTweetFeature.append(1)
            else:
                indiTweetFeature.append(0)

            print("Adding first person pronouns")
            #Contains pronoun first, sceond, third 7
            first_pronouns = ["i", "we"]
            tweetSplit = tweet.split(" ")
            for x in range(len(tweetSplit)):
                if(tweetSplit[x].lower() in first_pronouns):
                    indiTweetFeature.append(1)
                    break;
                if(x == len(tweetSplit)-1):
                    indiTweetFeature.append(0)

            print("Adding second person pronouns")
            second_pronouns = ["you"]
            tweetSplit = tweet.split(" ")
            for x in range(len(tweetSplit)):
                if(tweetSplit[x].lower() in second_pronouns):
                    indiTweetFeature.append(1)
                    break;
                if(x == len(tweetSplit)-1):
                    indiTweetFeature.append(0)

            print("Adding third person pronouns")
            third_pronouns = ["he", "him," "she", "her", "they", "them", "it"]

            tweetSplit = tweet.split(" ")
            for x in range(len(tweetSplit)):
                if(tweetSplit[x].lower() in third_pronouns):
                    indiTweetFeature.append(1)
                    break;
                if(x == len(tweetSplit)-1):
                    indiTweetFeature.append(0)

            print("Adding uppercase")
            #Count uppercase letters 8
            upperLetterCount = 0
            for letter in tweet:
                if(letter.isupper()):
                    upperLetterCount += 1
            indiTweetFeature.append(upperLetterCount)

            print("Adding url")
            #Number of URLs 9
            url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet) 
            indiTweetFeature.append(len(url))
        
            #Contains popular domain top 100,1000,10000
            print("Adding popular domain")
            file = pd.read_csv('/media/eric/TOSHIBA EXT/University/Research/Data/top-1m.csv')
            try:
                urls = tweet_json[count]['tweet']['entities']['urls']
            except:
                print(tweet_json[count]['tweet'])
            top100 = 0
            top1000 = 0
            top10000 = 0

            for url in urls:
                urlParsed = None
                rank = None
                try:
                    split_url = urlsplit(url['expanded_url'])   
                    shortUrl = ''
                    if(split_url.netloc[:4] == 'www.'):
                        shortUrl = split_url.netloc[4:]
                    else:
                        shortUrl = split_url.netloc
                    print(shortUrl)
                    print(file.loc[file['site'] == shortUrl]['rank'])
                    rank = int(file.loc[file['site'] == split_url.netloc]['rank'])
                except Exception as e:
                    rank = 10001
                if rank <= 100:
                    top100 = 1
                if rank <= 1000:
                    top1000 = 1
                if rank <= 10000:
                    top10000 = 1
        
            indiTweetFeature.append(top100)
            indiTweetFeature.append(top1000)
            indiTweetFeature.append(top10000)
                
            print("Adding mention")
            #Contains user mention 10
            result = re.findall("(^|[^@\w])@(\w{1,15})", tweet)
            if(len(result) >= threshold):
                indiTweetFeature.append(1)
            else:
                indiTweetFeature.append(0)
    
            # Length of hashtags 11
            print("Adding hashtags")
            hashtags = [i  for i in tweet.split() if i.startswith("#") ]
            indiTweetFeature.append(len(hashtags))
     
            #Contains stock symbol 12
            print("Addings stocks")
            file = open('/media/eric/TOSHIBA EXT/University/Research/Data/nasdaqlisted.txt', 'r')
            filecontents = file.readlines()
            stock_symbols = []
            for line in filecontents:
                splitline = line.split("|")
                stock_symbols.append(splitline[0])
            
            del stock_symbols[0]
        
            stockCount = 0
            for word in tweetSplit:
                if(word in stock_symbols):
                    stockCount += 1
        
            if(stockCount > 0):
                indiTweetFeature.append(1)
            else:
                indiTweetFeature.append(0)
                
            retweets, date = singleMessageFeatureExtraction(tweet_json[count])
            print("Adding retweets")
            indiTweetFeature.append(retweets)
            
            print("Adding date")
            indiTweetFeature = indiTweetFeature + date
        
            print("Adding sentiment")
            #Sentiment 13, 14, 15
            positiveWords = 0
            negativeWords= 0
    
    
            tweetSentiment = TextBlob(tweet)
            sentimentScore = tweetSentiment.sentiment[0]
    
            for word in tweetSentiment:
                word = TextBlob(word)
                if(word.sentiment.polarity) > 0:
                    positiveWords = positiveWords + 1
                elif(word.sentiment.polarity) < 0:
                    negativeWords = negativeWords + 1
            indiTweetFeature.append(positiveWords)
            indiTweetFeature.append(negativeWords)
            indiTweetFeature.append(sentimentScore)
        
        
            indiTweetFeature = [indiTweetFeature]
            if(count == 0):
                tweetsFeatures = indiTweetFeature
                print("Length of array: {}".format(len(tweetsFeatures[count])))
            else:
                tweetsFeatures = np.append(tweetsFeatures, indiTweetFeature, axis = 0)
                print(tweetsFeatures.shape)
        
            print("Finished feature extraction for tweet {} of {}".format(count, len(tweets)))
            count += 1
        
    return topicFeatures

In [14]:
def topicFeatureExtraction(topics):
    
    for topic in topics:
        
        urls = []
        
        for tweet in topic:
        print("Adding popular domain")
        file = pd.read_csv('/media/eric/TOSHIBA EXT/University/Research/Data/top-1m.csv')
        #Fraction Popular Domain Top 100, 1000, 10,000
        
        try:
            urls = tweet_json[count]['tweet']['entities']['urls']
        except:
            print(tweet_json[count]['tweet'])
        top100 = 0
        top1000 = 0
        top10000 = 0

        for url in urls:
            urlParsed = None
            rank = None
            try:
                urls.append(url['expanded_url'])
                split_url = urlsplit(url['expanded_url'])   
                shortUrl = ''
                if(split_url.netloc[:4] == 'www.'):
                    shortUrl = split_url.netloc[4:]
                else:
                    shortUrl = split_url.netloc
                print(shortUrl)
                print(file.loc[file['site'] == shortUrl]['rank'])
                rank = int(file.loc[file['site'] == split_url.netloc]['rank'])
            except Exception as e:
                rank = 10001
            if rank <= 100:
                top100 = 1
            if rank <= 1000:
                top1000 = 1
            if rank <= 10000:
                top10000 = 1
    
        #Count Distinct Seemingly Shortened URLs
    

    
        #Count Distinct Users Mentioned
    
        #Share most Frequent Users Mentioned
    
        #Count Distinct Authors
    
        #Share most Frequent Authors
    
        #Author Average Registration AGe
    
        #Author Average Statuses Count
    
        #Author Average Count Followers
    
        #Author Average Count Friends
    
        #Author Fraction is Verified
    
        #Author Fraction has Description
    
        #Author Fraction has URL
        
    #Count Distinct Expanded URLs
    
    #Share most Frequent Expanded URL
    
    #Count Distinct Hashtags
    
    #Share most Frequent Hashtag

In [15]:
def userFeatureExtraction():
    
    #Registration Age
    
    #Number of followers
    
    #Statuses Count (No. of Tweets)
    
    #Count Friends
    
    #Is Verified
    
    #Has Bio
    
    #Has URL

In [16]:
def propagationFeatureExtraction():
    
    #Propogation Initial Tweets
    
    #Propagation Max Subtree
    
    #Propagation Average Degree
    
    #Propagation Max Average Depth
    
    #Propagation Max Level